# Exploratory Data Analysis
## Haberman's Data set

Contains details of cancer survival rates

In [1]:
#Import required packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels import robust

In [5]:
#Read the data set
df = pd.read_csv('haberman.csv', names = ['Age','Op_Year','axil_nodes','Surv_status'])
df

,Age,Op_Year,axil_nodes,Surv_status
0,30,64,1,1
1,30,62,3,1
2,30,65,0,1
3,31,59,2,1
4,31,65,4,1
...,...,...,...,...
301,75,62,1,1
302,76,67,0,1
303,77,65,3,1
304,78,65,1,2
